In [2]:
import requests
from config.user import USERNAME, PASSWORD
from utils.optimizations import lazyproperty
        

class AuthentiseApiHandle:
    """
    A starter class to encapsulate some common
    interactions with Authentise's API(s)
    
    **NOTE** Assumes registration of user and
    that the user has an Authentise username and
    password
    """

    _3DIAX_RESOURCE_URLS = {
        'sessions': 'https://users.authentise.com/sessions/',
        'models': "https://models.authentise.com/model/",
    }
    
    def __init__(self, username, password):
        self._username = username
        self._password = password
        self._session, self._init_response = self._init_session()
    
    
    def _check_response(self, response):
        if response.ok:
            return True
        else:
            raise requests.HTTPError(
                "Bad response {}".format(response.text)
            )

            
    def _init_session(self, url=_3DIAX_RESOURCE_URLS['sessions']):
        """
        Returns a requests.Session() object if
        the supplied username and password were
        successfully authenticated
        """
        session = requests.Session()
        payload = {
            'username': self._username,
            'password': self._password,
        }
        response = session.post(url, json=payload)
        if self._check_response(response):
            return (session, response)

        
    def create_model_resource(self, model_name, 
                              url=_3DIAX_RESOURCE_URLS['models']):
        """
        Create a new model resource for
        the current session user
        """
        payload = {
            "name": model_name,
        }
        response = self._session.post(url, json=payload)
        if self._check_response(response):
            return response
        

    @lazyproperty
    def resources(self, url=_3DIAX_RESOURCE_URLS['models']):
        """
        Returns the the current model resource(s)
        for the current session user. One time compute
        to avoid hitting server too many times.
        """
        response = self._session.get(url)
        if self._check_response(response):
            # Convert JSON to Python dict, get resources based on 
            # current response schema response resources is a list with
            # a singular parent dictionary with various nested structures
            resources = response.json()
            self.resources_links = resources['links']
            self.resources_meta = resources['meta']
            return resources['resources'][0]


    def __repr__(self):
        return "{} active session for {}".format(
            self.__class__.__name__, 
            self._init_response.json()['name']
        )

In [3]:
# New instance
api_handle = AuthentiseApiHandle(USERNAME, PASSWORD)

In [4]:
# repr
api_handle

AuthentiseApiHandle active session for Erik

In [7]:
# Grab resources, this seems to be the 'try it out' default model and impervious
# to any new STL uploads. Just returning keys here, no values.

api_handle.resources.keys()

dict_keys(['analyses', 'callback', 'children', 'content', 'created', 'name', 'parents', 'rights', 'size', 'snapshot', 'snapshot_content', 'status', 'type', 'updated', 'upload-location', 'uri', 'user_uri', 'polygons', 'file_size', 'volume_mm', 'surface_area_mm', 'unit'])

In [9]:
api_handle.resources_links.keys()

dict_keys(['self', 'next', 'prev'])

In [11]:
api_handle.resources_meta.keys()

dict_keys(['count'])